<a href="https://colab.research.google.com/github/shreedharmalpani/soc_check/blob/main/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries

In [ ]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
import random

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/IMDB Dataset.csv')
print(df.columns)
df.describe()

Index(['review', 'sentiment'], dtype='object')


,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,positive
freq,5,25000


In [ ]:
def splitDF(r):
  dataLen = len(df)
  temp = ['test']*int((1-r)*dataLen) + ['train']*int((r)*dataLen)
  random.shuffle(temp)
  df['split'] = temp

  df.describe()

#splitDF(0.3)

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
df['review'] = df['review'].apply(lambda x:x.lower())

In [ ]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
df['review'] = df['review'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [ ]:
df['review'][0]

"one reviewers mentioned watching 1 oz episode hooked. right, exactly happened me.<br /><br />the first thing struck oz brutality unflinching scenes violence, set right word go. trust me, show faint hearted timid. show pulls punches regards drugs, sex violence. hardcore, classic use word.<br /><br />it called oz nickname given oswald maximum security state penitentary. focuses mainly emerald city, experimental section prison cells glass fronts face inwards, privacy high agenda. em city home many..aryans, muslims, gangstas, latinos, christians, italians, irish more....so scuffles, death stares, dodgy dealings shady agreements never far away.<br /><br />i would say main appeal show due fact goes shows dare. forget pretty pictures painted mainstream audiences, forget charm, forget romance...oz mess around. first episode ever saw struck nasty surreal, say ready it, watched more, developed taste oz, got accustomed high levels graphic violence. violence, injustice (crooked guards who'll sold

In [ ]:
def remove_punctuation(text):
    final = "".join(u for u in text if u not in ("?", ".", ";", ":", "!", '"', ',','#','$','@','%','^','&','*'))
    return final

df['review'] = df['review'].apply(remove_punctuation)

In [ ]:
def remove_tags(text):
    final = ""
    stt = True
    for char in text:
        if char == '<':
            stt = False
        if(stt):
            final = final + char
        if char == '>':
            stt = True
            final = final + ' '
    return final
    
df['review'] = df['review'].apply(remove_tags)

In [ ]:
df['review'][0]

"one reviewers mentioned watching 1 oz episode hooked right exactly happened me  the first thing struck oz brutality unflinching scenes violence set right word go trust me show faint hearted timid show pulls punches regards drugs sex violence hardcore classic use word  it called oz nickname given oswald maximum security state penitentary focuses mainly emerald city experimental section prison cells glass fronts face inwards privacy high agenda em city home manyaryans muslims gangstas latinos christians italians irish moreso scuffles death stares dodgy dealings shady agreements never far away  i would say main appeal show due fact goes shows dare forget pretty pictures painted mainstream audiences forget charm forget romanceoz mess around first episode ever saw struck nasty surreal say ready it watched more developed taste oz got accustomed high levels graphic violence violence injustice (crooked guards who'll sold nickel inmates who'll kill order get away it well mannered middle class 

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
df['review'] = df['review'].apply(nltk.word_tokenize)

In [ ]:
def stem_tokens(tokens):
    final = [nltk.stem.PorterStemmer().stem(word) for word in tokens]
    return final
    
df['review'] = df['review'].apply(stem_tokens)

In [ ]:
df['len_review'] = df['review'].apply(lambda x:len(x))

In [ ]:
df['sentiment'] = [1*(sent=='positive') for sent in df['sentiment']]

In [ ]:
df.head(4)

,review,sentiment,len_review
0,"[one, review, mention, watch, 1, oz, episod, h...",1,175
1,"[wonder, littl, product, the, film, techniqu, ...",1,96
2,"[thought, wonder, way, spend, time, hot, summe...",1,95
3,"[basic, there, 's, famili, littl, boy, (, jake...",0,74


In [ ]:
from gensim import corpora

review_dict = corpora.Dictionary(df['review'])
VOCAB_SIZE = len(review_dict)

In [ ]:
review_dict?

In [ ]:
print(len(review_dict))
review_dict

132056


In [ ]:
review_int = []
for review in df['review']:
  temp = [review_dict.token2id[x] for x in review]
  review_int.append(temp)
df['review_int'] = review_int

In [ ]:
df.head(4)

,review,sentiment,len_review,review_int
0,"[one, review, mention, watch, 1, oz, episod, h...",1,175,"[90, 104, 80, 137, 3, 93, 33, 60, 105, 35, 55,..."
1,"[wonder, littl, product, the, film, techniqu, ...",1,96,"[206, 171, 183, 126, 162, 196, 202, 175, 161, ..."
2,"[thought, wonder, way, spend, time, hot, summe...",1,95,"[265, 206, 269, 256, 266, 230, 260, 271, 255, ..."
3,"[basic, there, 's, famili, littl, boy, (, jake...",0,74,"[281, 314, 144, 291, 171, 283, 1, 295, 2, 315,..."


In [ ]:
review_int = df['review_int']
review_len = df['len_review']
review_label = df['sentiment']
review_int = [review_int[i] for i,x in enumerate(review_int) if review_len[i]>0]
review_label = [review_label[i] for i,x in enumerate(review_label) if review_len[i]>0]
review_len = [review_len[i] for i,x in enumerate(review_len) if review_len[i]>0 ]

In [ ]:
len(review_int)

50000

In [ ]:
def pad_features(review_int, L):
  features = np.zeros((len(review_int),L), dtype = int)
  for i,review in enumerate(review_int):
    l = len(review)
    if l <= L:
      patch = [0]*(L-l)
      new = patch + review
      features[i,:] = np.array(new)
    else :
      features[i,:] = np.array(review[:L])
  return features

In [ ]:
features = pad_features(review_int, 200)

In [ ]:
print(features[:10,:])

[[  0   0   0 ... 129  24 117]
 [  0   0   0 ... 197 138 154]
 [  0   0   0 ...  50 190 227]
 ...
 [  0   0   0 ... 404 258 213]
 [  0   0   0 ... 556 561 562]
 [  0   0   0 ...  66 165 571]]


In [ ]:
split_frac = 0.8

train_x = features[0:int(len(features)*split_frac)]
train_y = np.array(review_label[0:int(len(features)*split_frac)])

test_x = features[len(features)-int(len(features)*split_frac):len(features)-int(len(features)*split_frac/2)]
test_y = np.array(review_label[len(features)-int(len(features)*split_frac):len(features)-int(len(features)*split_frac/2)])

valid_x = features[len(features)-int(len(features)*split_frac/2):]
valid_y = np.array(review_label[len(features)-int(len(features)*split_frac/2):])

In [ ]:
from torch.utils.data import DataLoader, TensorDataset

train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
test_data = TensorDataset(torch.from_numpy(test_x), torch.from_numpy(test_y))
valid_data = TensorDataset(torch.from_numpy(valid_x), torch.from_numpy(valid_y))

batch_size = 50

train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size)

In [ ]:
# obtain one batch of training data
dataiter = iter(train_loader)
sample_x, sample_y = dataiter.next()
print('Sample input size: ', sample_x.size()) # batch_size, seq_length
print('Sample input: \n', sample_x)
print()
print('Sample label size: ', sample_y.size()) # batch_size
print('Sample label: \n', sample_y)

Sample input size:  torch.Size([50, 200])
Sample input: 
 tensor([[    0,     0,     0,  ...,  7912,  4062,   299],
        [    0,     0,     0,  ..., 31716,  4147, 13425],
        [    0,     0,     0,  ...,  6921,   512,    90],
        ...,
        [    0,     0,     0,  ...,   538,   144,   344],
        [    0,     0,     0,  ..., 43965,    90,   938],
        [    0,     0,     0,  ..., 52949,  9128,  2709]])

Sample label size:  torch.Size([50])
Sample label: 
 tensor([0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1,
        1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0,
        1, 1])


In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device available for running: ")
print(device)

Device available for running: 
cuda


In [ ]:
class SentimentRNN(nn.Module):
    """
    The RNN model that will be used to perform Sentiment analysis.
    """

    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):
        """
        Initialize the model by setting up the layers.
        """
        super().__init__()

        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_dim, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        self.dropout = nn.Dropout(0.3)
        
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sig = nn.Sigmoid()
        

    def forward(self, x, hidden):
      
        batch_size = x.size(0)
  
        embeds = self.embedding(x)
        rnn_out, hidden = self.rnn(embeds, hidden)
    
        rnn_out = rnn_out.contiguous().view(-1, self.hidden_dim)
        
        out = self.dropout(rnn_out)
        out = self.fc(out)
        sig_out = self.sig(out)
        
        sig_out = sig_out.view(batch_size, -1)
        sig_out = sig_out[:, -1] 
        
        return sig_out, hidden
    
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device)
        return hidden
        

In [ ]:

vocab_size = len(review_dict)+1
output_size = 1
embedding_dim = 400
hidden_dim = 256
n_layers = 2
net = SentimentRNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)

print(net)

SentimentRNN(
  (embedding): Embedding(132057, 400)
  (rnn): RNN(400, 256, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=256, out_features=1, bias=True)
  (sig): Sigmoid()
)


In [ ]:
train_on_gpu=torch.cuda.is_available()

if(train_on_gpu):
    print('Training on GPU.')
else:
    print('No GPU available, training on CPU.')

Training on GPU.


In [ ]:
lr=0.001

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)

In [ ]:


epochs = 10

counter = 0
print_every = 100
clip=5 

if(train_on_gpu):
    net.cuda()

net.train()
for e in range(epochs):
    h = net.init_hidden(batch_size)

    for inputs, labels in train_loader:
        counter += 1
       
        if(train_on_gpu):
            inputs, labels = inputs.cuda(), labels.cuda()

      
        h = h.data
        net.zero_grad()

        output, h = net(inputs, h)

        loss = criterion(output.squeeze(), labels.float())
        loss.backward()
        nn.utils.clip_grad_norm_(net.parameters(), clip)
        optimizer.step()

        if counter % print_every == 0:
            val_h = net.init_hidden(batch_size)
            val_losses = []
            net.eval()
            for inputs, labels in valid_loader:

                val_h = h.data

                if(train_on_gpu):
                    inputs, labels = inputs.cuda(), labels.cuda()

                output, val_h = net(inputs, val_h)
                val_loss = criterion(output.squeeze(), labels.float())

                val_losses.append(val_loss.item())

            net.train()
            print("Epoch: {}/{}...".format(e+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))

Epoch: 1/10... Step: 100... Loss: 0.703180... Val Loss: 0.604725
Epoch: 1/10... Step: 200... Loss: 0.526134... Val Loss: 0.609699
Epoch: 1/10... Step: 300... Loss: 0.668048... Val Loss: 0.615257
Epoch: 1/10... Step: 400... Loss: 0.588628... Val Loss: 0.574451
Epoch: 1/10... Step: 500... Loss: 0.595302... Val Loss: 0.595147
Epoch: 1/10... Step: 600... Loss: 0.681395... Val Loss: 0.656499
Epoch: 1/10... Step: 700... Loss: 0.673461... Val Loss: 0.699640
Epoch: 1/10... Step: 800... Loss: 0.607146... Val Loss: 0.600760
Epoch: 2/10... Step: 900... Loss: 0.675718... Val Loss: 0.522857
Epoch: 2/10... Step: 1000... Loss: 0.605045... Val Loss: 0.562030
Epoch: 2/10... Step: 1100... Loss: 0.479951... Val Loss: 0.552487
Epoch: 2/10... Step: 1200... Loss: 0.505550... Val Loss: 0.528833
Epoch: 2/10... Step: 1300... Loss: 0.571972... Val Loss: 0.513054
Epoch: 2/10... Step: 1400... Loss: 0.632299... Val Loss: 0.523574
Epoch: 2/10... Step: 1500... Loss: 0.528826... Val Loss: 0.508642
Epoch: 2/10... Step

In [ ]:


test_losses = [] 
num_correct = 0

h = net.init_hidden(batch_size)

net.eval()
for inputs, labels in test_loader:

    h = h.data

    if(train_on_gpu):
        inputs, labels = inputs.cuda(), labels.cuda()
    
    output, h = net(inputs, h)
    
    test_loss = criterion(output.squeeze(), labels.float())
    test_losses.append(test_loss.item())
    
    pred = torch.round(output.squeeze())  
    
    
    correct_tensor = pred.eq(labels.float().view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    num_correct += np.sum(correct)



print("Test loss: {:.3f}".format(np.mean(test_losses)))


test_acc = num_correct/len(test_loader.dataset)
print("Test accuracy: {:.3f}%".format(test_acc*100))

Test loss: 0.285
Test accuracy: 89.505%


In [ ]:
def preprocess(review):
    review = review.lower()
    word_list = review.split()
    num_list = []
    
    reviews_int = []
    for word in word_list:
        if word in review_dict.token2id:
            num_list.append(review_dict.token2id[word])
    reviews_int.append(num_list)
    return reviews_int

In [ ]:
def predict(net, test_review, sequence_length=200):
    
    int_rev = preprocess(test_review)
    features = pad_features(int_rev, L=seq_length)
    
    features = torch.from_numpy(features)
    
    net.eval()
    val_h = net.init_hidden(1)
    val_h = val_h.data

    if(train_on_gpu):
        features = features.cuda()

    output, val_h = net(features, val_h)
    
    pred = torch.round(output)
    
    sent = ["Positive" if pred.item() == 1 else "Negative"]
    
    print(sent,'  ',((pred.item()==1)*output.item()+(pred.item()==0)*(1-output.item()))*100,'%')

In [ ]:
positive_review = 'A great movie. Totally worth every penny. '

In [ ]:
seq_length=200
predict(net, positive_review, seq_length)

['Positive']    93.45735907554626 %


In [ ]:
negative_review = 'Amongst the worst works of diresctor nick furry. I want my money back. Waste of time.'

In [ ]:
seq_length=200
predict(net, negative_review, seq_length)

['Negative']    86.78118586540222 %


In [ ]:


test_losses = [] 
num_correct = 0

h = net.init_hidden(batch_size)

net.eval()
for inputs, labels in valid_loader:

    
    h = h.data

    if(train_on_gpu):
        inputs, labels = inputs.cuda(), labels.cuda()
    
    output, h = net(inputs, h)
    
    test_loss = criterion(output.squeeze(), labels.float())
    test_losses.append(test_loss.item())
    
    pred = torch.round(output.squeeze()) 
    
    correct_tensor = pred.eq(labels.float().view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    num_correct += np.sum(correct)



print("Test loss: {:.3f}".format(np.mean(test_losses)))


test_acc = num_correct/len(valid_loader.dataset)
print("Test accuracy: {:.3f}%".format(test_acc*100))

Test loss: 0.383
Test accuracy: 87.360%
